In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import os
from functools import partial
from multiprocessing import Pool

from fastdtw import fastdtw, dtw
from sklearn import svm
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from sklearn.metrics import confusion_matrix  

In [2]:
reduced = pd.read_csv("reduced.csv", index_col="date")
d = reduced.apply(np.log).diff()[1:]
data = []
for podatak in d["HP"]:
    data.append(podatak)

In [3]:
L = len(data)
l = 20
s = 1

X = np.stack([data[i:i + l] for i in range(L // 2 - l - s)])
Y = np.stack([data[i + l + s - 1] > data[i + l - 1] for i in range(L // 2 - l - s)]).astype(np.float32)
##ako je sljedeći element veći od zadnjeg clana niza onda dobiva 1 inace 0

X_test = np.stack([data[i:i + l] for i in range(L // 2, L - l - s)])
Y_test = np.stack([data[i + l + s - 1] > data[i + l - 1] for i in range(L // 2, L - l - s)]).astype(np.float32)


In [4]:
len(X), len(X_test), Y.mean(), Y_test.mean()

(463, 464, 0.4838013, 0.51939654)

In [5]:
n = len(X)
m = len(X_test)
kernel_train = np.empty((n, n))
kernel_test = np.empty((m, n))

In [6]:
t0 = time.time()
for i in range(n):
    for j in range(i+1):
        if i == j:
            kernel_train[i][j] = 0.0
        else:
            kernel_train[i][j] = kernel_train[j][i] = fastdtw(X[i], X[j], radius = 5, dist = 2)[0]


for i in range(m):
    for j in range(n):           
        kernel_test[i][j] = fastdtw(X_test[i], X[j], radius = 5, dist = 2)[0]
                      
print(time.time()-t0)

11424.631430387497


In [9]:
#np.save("kernel_train_univariant_HP.npy", kernel_train)

In [10]:
kernel_train = np.load("kernel_train_univariant_HP.npy")

In [11]:
#np.save("kernel_test_univariant_HP.npy", kernel_test)

In [12]:
kernel_test = np.load("kernel_test_univariant_HP.npy")

In [14]:
c = 10
gamma = 25
kernel = np.exp(-kernel_train/gamma)
testkernel = np.exp(-kernel_test/gamma)
clf = svm.SVC(kernel='precomputed', C=c)
clf.fit(kernel, Y)
res = clf.predict(testkernel)
print('accuracy test: ', np.mean(res == Y_test))
print('Area under curve(AUC) test:', roc_auc_score(Y_test, res))

accuracy test:  0.7974137931034483
Area under curve(AUC) test: 0.7976108516457957


In [15]:
res = clf.predict(kernel)
print('accuracy train: ', np.mean(res == Y))
print('Area under curve(AUC) train:', roc_auc_score(Y, res))

accuracy train:  0.7688984881209503
Area under curve(AUC) train: 0.7681653466826062


In [7]:
#za razlicite konstante
#for c in range(6,14):
#    for gamma in range(24, 32):
#        kernel = np.exp(-kernel_train/gamma)
#        testkernel = np.exp(-kernel_test/gamma)
#        clf = svm.SVC(kernel='precomputed', C=c)
#        clf.fit(kernel, Y)
#        res = clf.predict(testkernel)
#        print('C:{}, gamma:{}, accuracy:{} '.format(c, gamma, np.mean(res == Y_test)))